In [ ]:
%pip install pandas

import pandas as pd

full_df = pd.read_csv('delaney_descriptors.csv', index_col=0)
full_df.describe()

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'pandas'

### Load FASTA database
Parse the local FASTA export into a DataFrame so downstream code can work with accession, description, and sequence fields.

In [ ]:
from pathlib import Path

fasta_path = Path("uniprotkb_accession_D4A7P2_OR_accession_2025_12_08.fasta")

def _build_record(header, seq_lines):
    full_header = header.strip()
    accession, *desc = full_header.split(maxsplit=1)
    return {
        "header": full_header,
        "accession": accession,
        "description": desc[0] if desc else "",
        "sequence": "".join(seq_lines),
    }

def load_fasta(path: Path):
    records = []
    header = None
    seq_lines = []

    with path.open() as handle:
        for raw_line in handle:
            line = raw_line.strip()
            if not line:
                continue
            if line.startswith(">"):
                if header:
                    records.append(_build_record(header, seq_lines))
                header = line[1:]
                seq_lines = []
            else:
                seq_lines.append(line)

    if header:
        records.append(_build_record(header, seq_lines))

    return pd.DataFrame(records)

fasta_df = load_fasta(fasta_path)
print(f"Loaded {len(fasta_df)} sequences from {fasta_path}")
fasta_df.head()